In [26]:
import heapq
import math

In [27]:
GOAL = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

In [28]:
def is_solvable(state):
    flat = [tile for row in state for tile in row if tile != 0]
    inv = 0
    for i in range(len(flat)):
        for j in range(i + 1, len(flat)):
            if flat[i] > flat[j]:
                inv += 1
    return inv % 2 == 0

In [29]:
def manhattan(state):
    distance = 0
    for r in range(3):
        for c in range(3):
            tile = state[r][c]
            if tile != 0:
                goal_r, goal_c = divmod(tile - 1, 3)
                distance += abs(r - goal_r) + abs(c - goal_c)
    return distance

In [30]:
def neighbors(state):
    results = []
    for r in range(3):
        for c in range(3):
            if state[r][c] == 0:
                blank_r, blank_c = r, c
                break

    moves = [(-1, 0, "Up"), (1, 0, "Down"), (0, -1, "Left"), (0, 1, "Right")]
    for dr, dc, name in moves:
        nr, nc = blank_r + dr, blank_c + dc
        if 0 <= nr < 3 and 0 <= nc < 3:
            new_state = [row[:] for row in state]
            new_state[blank_r][blank_c], new_state[nr][nc] = new_state[nr][nc], new_state[blank_r][blank_c]
            results.append((new_state, name))
    return results

In [31]:
def reconstruct_path(came_from, current):
    path = []
    while came_from[str(current)][0] is not None:
        prev, move = came_from[str(current)]
        path.append(move)
        current = prev
    path.reverse()
    return path

In [32]:
def a_star(start):
    if start == GOAL:
        return [], 0, 0
    if not is_solvable(start):
        raise ValueError("Not solvable.")

    open_heap = []
    g_score = {str(start): 0}
    f_score = manhattan(start)
    heapq.heappush(open_heap, (f_score, 0, start))
    came_from = {str(start): (None, None)}
    closed_set = set()

    tie = 1

    while open_heap:
        current_f, _, current = heapq.heappop(open_heap)
        if str(current) in closed_set:
            continue
        if current == GOAL:
            moves = reconstruct_path(came_from, current)
            return moves

        closed_set.add(str(current))

        for nbr, move in neighbors(current):
            if str(nbr) in closed_set:
                continue
            tentative_g = g_score[str(current)] + 1
            if tentative_g < g_score.get(str(nbr), math.inf):
                came_from[str(nbr)] = (current, move)
                g_score[str(nbr)] = tentative_g
                f = tentative_g + manhattan(nbr)
                tie += 1
                heapq.heappush(open_heap, (f, tie, nbr))

    raise RuntimeError("No solution found.")

In [33]:
def pretty_print(state):
    for row in state:
        print(' '.join(str(x) if x != 0 else '_' for x in row))
    print()

if __name__ == "__main__":
    start_state = [
        [1, 2, 3],
        [4, 6, 7],
        [0, 5, 8]
    ]

    print("Start:")
    pretty_print(start_state)
    print("Goal:")
    pretty_print(GOAL)

    try:
        moves = a_star(start_state)
        print(f"Solution in {len(moves)} moves.")
        print("Moves:", moves)

        cur = start_state
        for i, m in enumerate(moves, 1):
            for nbr, mv in neighbors(cur):
                if mv == m:
                    cur = nbr
                    break
            print(f"After move {i}: {m}")
            pretty_print(cur)

    except ValueError as e:
        print(e)

Start:
1 2 3
4 6 7
_ 5 8

Goal:
1 2 3
4 5 6
7 8 _

Solution in 14 moves.
Moves: ['Up', 'Right', 'Right', 'Down', 'Left', 'Up', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Down', 'Right']
After move 1: Up
1 2 3
_ 6 7
4 5 8

After move 2: Right
1 2 3
6 _ 7
4 5 8

After move 3: Right
1 2 3
6 7 _
4 5 8

After move 4: Down
1 2 3
6 7 8
4 5 _

After move 5: Left
1 2 3
6 7 8
4 _ 5

After move 6: Up
1 2 3
6 _ 8
4 7 5

After move 7: Left
1 2 3
_ 6 8
4 7 5

After move 8: Down
1 2 3
4 6 8
_ 7 5

After move 9: Right
1 2 3
4 6 8
7 _ 5

After move 10: Right
1 2 3
4 6 8
7 5 _

After move 11: Up
1 2 3
4 6 _
7 5 8

After move 12: Left
1 2 3
4 _ 6
7 5 8

After move 13: Down
1 2 3
4 5 6
7 _ 8

After move 14: Right
1 2 3
4 5 6
7 8 _

